In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
def plot_history(log_path="experiments/exp_0", add_title=False):
    _, ax = plt.subplots()
    df_history = pd.read_csv(f"{log_path}/ga_history.csv")
    df_history['cumulative_average'] = df_history['best_cost'].expanding().mean()
    ax.plot(df_history['generation'], df_history['cumulative_average'], label='Cumulative Average of Best Cost')
    ax.plot(df_history['generation']-1, df_history['best_cost'], label='Best Cost per Generation', zorder=5)

    df_evaluations = pd.read_csv(f"{log_path}/ga_evaluations.csv")
    ax.scatter(df_evaluations['generation'], df_evaluations['evaluation_cost'], color='red', alpha=0.2, s=2, label='Offspring Evaluation')

    if add_title:
        json_path = f"{log_path}/ga_params.json"
        with open(json_path, "r") as f:
            params = json.load(f)
        improvement_over_baseline = df_history['best_cost'].min() / params["baseline_fitness"]
        title = f"n={params['problem_size']}, alpha={params['problem_alpha']}, beta={params['problem_beta']}, density={params['problem_density']:.1f}, i={improvement_over_baseline:.2f}"
        ax.set_title(title)

    ax.set_xlabel('Generation')
    ax.set_ylabel('Cost')
    ax.legend()
    plt.show()
    return ax

In [ ]:
for i in range(9):
    log_dir = f"experiments/exp_{i}"
    ax = plot_history(log_path=log_dir, add_title=True)

In [ ]:
data = []
for i in range(9):
    log_dir = f"experiments/exp_{i}"
    with open(f"{log_dir}/ga_params.json", "r") as f:
        params = json.load(f)
    df_history = pd.read_csv(f"{log_dir}/ga_history.csv")
    best_cost = df_history['best_cost'].min()
    params['best_cost'] = best_cost
    data.append(params)
df = pd.DataFrame(data)
df["improvement"] = df["best_cost"] / df["baseline_fitness"]
problem_features = ["problem_size", "problem_alpha", "problem_beta", "problem_density"]
others = ["population_size", "max_generations", "mutation_rate", "mutation_choice"]
df = df[problem_features + others + ["baseline_fitness", "best_cost", "improvement"]]

In [ ]:
df.rename(columns={
    "max_generations": "$n_g$",
    "mutation_rate": "$p_m$",
    "mutation_choice": "$p_s$",
    "problem_size": "$n$",
    "population_size": "$n_p$",
    "problem_alpha": "$\\alpha$",
    "problem_beta": "$\\beta$",
    "problem_density": "$\\rho$",
    "baseline_fitness": "cost (baseline)",
    "best_cost": "cost (GA)",
    "improvement": "improvement"
}, inplace=True)
table = df.to_latex(float_format="%.2f", index=False)
print(table)

In [ ]:
from Problem import Problem
p = Problem(num_cities=10, alpha=1, beta=1, density=0.2, seed=42)
p.plot()